# On pandas and ducks

- toc: true
- branch: master
- badges: false
- comments: true
- categories: [pandas, database]
- hide: false
- search_exclude: false
- annotations: true

> We use DuckDB for high-performance aggregates on Pandas dataframes. 

## The problem

In many analytical problems, we need to perform aggregates where the aggregation key is of very high cardinality. For example, in website actvity data, we might be interested in the duration of activity for each user. So we might perform a `group-by` operation on user_id, which can result in a very large number of groups, assuming there are many users.

### Simulate data

Below we generate 10 million data points, where each user_id has on average ten records.

In [25]:
import numpy as np
import pandas as pd

n_items = 10_000_000
n_keys = n_items // 10

dates = pd.date_range("2020-01-01 05:11", "2020-01-03 17:22", periods=1000)

df = pd.DataFrame({
    "user_id": np.random.choice(n_keys, size=n_items), 
    "ts":      np.random.choice(dates, size=n_items),
})

df.sample(2)

,user_id,ts
8195914,290816,2020-01-02 03:42:51.951951951
8511040,817826,2020-01-03 14:32:06.786786786


In [26]:
df["user_id"].nunique()

999951

### Pandas aggregation

Let's try to find the interval (in seconds) between first and last timestamp for each user using `pandas`.

In [27]:
def interval_seconds(s):
    return (s.max() - s.min()).total_seconds()

In [28]:
%%time

df.groupby("user_id")["ts"].agg(interval_seconds).max()

CPU times: user 2min 58s, sys: 5.7 s, total: 3min 3s
Wall time: 2min 57s


216660.0

This computation takes just under 3 minutes on my machine. When the number of group keys is very large, pandas does not perform well. Let's see if we can do better.

## DuckDB for the aggregation

[DuckDB](https://duckdb.org/) is an in-process column-based database. It is dubbed "SQlLite for analytics" because it is embedded into the application - there is no separate database server to be managed. It promises great performance for analytical queries on large datasets.

More specifically for our use-case, there is a strong integration between DuckDB and Pandas. 

In [29]:
import duckdb

# connect to an in-memory database
con = duckdb.connect()

In [30]:
%%time

query = """
with deltas AS (
    select 
      user_id, 
      extract('epoch' from max(ts) - min(ts)) AS delta_seconds
    from df
    group by user_id
)
    select max(delta_seconds) AS m
    FROM deltas
"""
results = con.execute(query).fetchone()
results[0]

CPU times: user 6.12 s, sys: 799 ms, total: 6.92 s
Wall time: 909 ms


216660

We perform a query directly on a pandas dataframe, without inserting it into the database first. This works because DuckDB uses Apache Arrow to transfer data efficiently with pandas.

The DuckDB query completes in less than a second, or approximately 180 times faster.

### Get the aggregation result back as a dataframe

In the example above, we return a single number, the max duration. But what if we wanted to get all durations back as a pandas dataframe? Maybe we want to plot the results, or continue the analysis. Fortunately, this is also easy to do.

In [31]:
%%time

query = """
    select 
      user_id, 
      extract('epoch' from max(ts) - min(ts)) AS delta_seconds
    from df
    group by user_id
"""
result_df = con.execute(query).df()
result_df.head(2)

CPU times: user 5.46 s, sys: 8.26 ms, total: 5.47 s
Wall time: 776 ms


,user_id,delta_seconds
0,652093,174369
1,308985,200394


## Does the speed-up really matter?

We saw some impressive speed-up when using DuckDB for the aggregation. But we must ask ourselves if this speed-up really impacts our work. 

I like to divide computational tasks into the following categories.

* Interactive work: < 1 minute
* Slower tasks: < 1 hour
* Batch tasks: more than 1 hour, could be days.

### Interactive workloads
Interactive workloads finish within a minute or so. Typically, people are most productive in this category because they can pose a question (in the form of a query or computational task) and get answers very quickly. They can follow their train of thoughts without major interruptions or distractions.

### Slower tasks

### Batch tasks
Batch tasks require a significant amount of time and computation.